In [1]:
import pandas as pd


# Read the cleaned file into a DataFrame
df_cleaned = pd.read_csv('../data/bgg_rating_threshold.csv', sep=';', engine='c')

# Convert the 'bgg_user_rating' column to numeric
df_cleaned['bgg_user_rating'] = pd.to_numeric(df_cleaned['bgg_user_rating'], errors='coerce')

print("done")

done


In [ ]:
import numpy as np
import scipy
# Create the matrix with games as rows and users as columns.
df_cleaned['bgg_user_rating'] = (df_cleaned['bgg_user_rating']*10).astype(np.uint8)
#user_game_matrix = df_cleaned.head(1000000).pivot(index='bgg_id', columns='bgg_user_name', values='bgg_user_rating')
user_game_matrix = df_cleaned.set_index(['bgg_id', 'bgg_user_name'])['bgg_user_rating'].unstack('bgg_user_name',0)
sparse = scipy.sparse.csr_matrix(user_game_matrix.values)
# Check the shape and preview the matrix.
print(user_game_matrix.shape)

/tmp/ipykernel_24348/1491942108.py:5: PerformanceWarning: The following operation may generate 6489018225 cells in the resulting pandas object.
  user_game_matrix = df_cleaned.set_index(['bgg_id', 'bgg_user_name'])['bgg_user_rating'].unstack('bgg_user_name',0)


(15751, 411975)


In [ ]:
import numpy as np
import scipy
from sklearn.decomposition import NMF


model = NMF(n_components=600, verbose=True)


W = model.fit_transform(sparse)
print(model.reconstruction_err_)
print(model.n_iter_)
#np.savetxt("../data/features.csv", u4, delimiter=";")

violation: 1.0
violation: 0.427093889198277
violation: 0.17961674399362065
violation: 0.14603857183056737
violation: 0.12041737770242726
violation: 0.09997198379369744
violation: 0.08377027290138427
violation: 0.07085235799376131
violation: 0.0606353881071597
violation: 0.052432465051327384
violation: 0.045617529334151447
violation: 0.039834948283098456
violation: 0.034858467143574295
violation: 0.030560020149855996
violation: 0.026836580463178158
violation: 0.02359109815400676
violation: 0.02076321558842384
violation: 0.018308363164916706
violation: 0.016153936571178595
violation: 0.014251836028635254
violation: 0.012570453599363722
violation: 0.011097228564029818
violation: 0.009802923228250528
violation: 0.008664292819749161
violation: 0.007661321732771349
violation: 0.006778264791536119
violation: 0.005998401522144323
violation: 0.005310108000040205
violation: 0.004702224092185337
violation: 0.0041655082489777595
violation: 0.0036906434345398546
violation: 0.003270477512523504
viol

In [ ]:
# print(W.shape)
# np.savetxt("../data/features_100.csv", W, delimiter=";")
# df_cleaned['bgg_id'].to_csv("../data/bgg_id.csv", sep=";", index=True)

(15751, 600)


In [9]:
# Compute the Frobenius norm of the difference.
error = model.n_iter_

# And the Frobenius norm of the original matrix.
norm_orig = scipy.sparse.linalg.norm(sparse, ord='fro')

relative_error = error / norm_orig
print("Relative Frobenius error:", relative_error)

Relative Frobenius error: 0.0001698427591223435
